## Capstone Project

### Modeling Notebook - `Polynomial LogisticRegression`  `COMBINED DATASET` from 10/24/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/Users/nate_velarde/anaconda/envs/py3jp/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.1
Running numpy version: 1.14.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 '273_vs_281_null_count_by_feature.csv',
 '273_vs_281_null_count_by_feature.xlsm',
 'Capstone - Complication list - complete.xlsx',
 'Capstone - STS risk factor list.xlsx',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_10_27.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'PREOP_dataset_TREE_10_27.pkl',
 'X_A_DREF.pkl',
 'X_A_DREF_TREE_SKLEARN.pkl',
 'X_PREOP_10_24.pkl',
 'X_PREOP_10_27.pkl',
 'X_PREOP_TREE_10_24.pkl',
 'X_PREOP_TREE_10_27.pkl',
 'X_dev_A_DREF.pkl',
 'X_dev_A_DREF_TREE_SKLEARN.pkl',
 'X_dev_PREOP_10_24.pkl',
 'X_dev_PREOP_10_27.pkl',
 'X_dev_PREOP_TREE_10_24.pkl',
 'X_dev_PREOP_TREE_10_27.pkl',
 'X_dev_PREOP_TREE_UNPROC_10_27.pkl',
 'X_dev_PREOP_UNPROC_10_24.pkl',
 'X_dev_PREOP_UNPROC_10_27.pkl',
 'X_test_A_DREF.pkl',
 'X_test_A_DREF_TREE_SKLEARN.pkl',
 'X_test_PREOP_10_24.pkl',
 'X_test_PREOP_10_27.pkl',
 'X_test_PREOP_TREE_10_24.pkl',
 'X_test_PREOP_TREE_10_27.pkl',
 'X_test_PREOP_TREE_UNPROC_10_27.pkl',
 'X_test_PRE

### Loading Datasets
- Going to load the unprocessed versions of our data in order to run through pipelines

#### `X_train`, `y_train`
- designation of `_all` denotes complete feature set

In [7]:
X_train_all = pd.read_pickle('X_train_PREOP_UNPROC_10_27.pkl')
y_train = pd.read_pickle('y_train_PREOP_UNPROC_10_27.pkl')

In [8]:
X_train_all.shape, y_train.shape

((34192, 110), (34192,))

#### `X_dev`, `y_dev`
- designation of `_all` denotes complete feature set

In [9]:
X_dev_all = pd.read_pickle('X_dev_PREOP_UNPROC_10_27.pkl')
y_dev = pd.read_pickle('y_dev_PREOP_UNPROC_10_27.pkl')

In [10]:
X_dev_all.shape, y_dev.shape

((4274, 110), (4274,))

#### `X_test`, `y_test`
- designation of `_all` denotes complete feature set

In [11]:
X_test_all = pd.read_pickle('X_dev_PREOP_UNPROC_10_27.pkl')
y_test = pd.read_pickle('y_dev_PREOP_UNPROC_10_27.pkl')

In [12]:
X_test_all.shape, y_dev.shape

((4274, 110), (4274,))

- validating row count for `COMBINED DATASET` from 10/24/19 - `42,740` total observations

In [13]:
42740 - X_train_all.shape[0] - X_dev_all.shape[0] - X_test_all.shape[0]

0

- last look at the data (`X_train_all`) before modeling

In [14]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,43,172.7,96.2,32.25451,24.4,0.81,2.6,4.1,15.42,65.0,71.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,78,162.5,69.9,26.47101,38.2,1.01,NaN,6.7,NaN,57.0,41.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,64,188.0,121.5,34.37641,42.0,0.90,3.6,6.8,6.40,60.0,NaN,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,71,168.0,85.0,30.11621,42.0,0.90,NaN,5.3,NaN,55.0,30.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,58,160.0,93.4,36.48438,27.0,0.80,NaN,6.9,NaN,60.0,29.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [15]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
34187,63,177.8,105.6,33.40415,39.0,0.90,NaN,7.5,NaN,53.0,41.7,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
34188,54,178.0,86.4,27.26928,36.0,0.70,2.7,7.2,10.00,14.0,52.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
34189,73,160.0,93.0,36.32812,37.0,1.07,3.6,4.9,7.05,70.0,NaN,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34190,77,155.0,105.0,43.70447,37.7,0.70,4.1,7.8,6.40,60.0,56.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34191,64,178.0,104.0,32.82414,35.0,1.10,3.7,10.7,7.30,63.0,27.0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## Pre-processing Data using `Scikit-Learn` `Pipelines`

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data (numerical and categorical features) -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [16]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

- create `lists` of `numerical_features` and `categorical_features` to use to mask/select features from dataset

In [17]:
numerical_features = ['age',
                      'heightcm',
                      'weightkg',
                      'bmi',
                      'hct',
                      'creatlst',
                      'totalbumin',
                      'a1clvl',
                      'meldscr',
                      'hdef',
                      'pasys']

- `categorical_features` are the subset of the whole feature set that are `not in` `numerical_features`
- use `np.setdiff1d` to identify elements in one list that are not in another

In [18]:
categorical_features = list(np.setdiff1d(X_train_all.columns.tolist(), 
                                        numerical_features,
                                        assume_unique=True))

In [19]:
print (len(numerical_features))
print (len(categorical_features))
print (len(numerical_features) + len(categorical_features))
print (X_train_all.shape[1])

11
99
110
110


## Functions Used in Pipelines

#### `SimpleImputer(missing_values=np.nan, strategy='median'` 
- to replace numerical feature `NaN`s with `X_train` numerical feature `NaN`s

#### `PolynomialFeatures()`
- per Albon 13.2 pages 225-227, can use `PolynomialFeatures()` to create interaction terms
- Parameter `interaction_only=True` tells `PolynomialFeatures()` to ONLY return interaction terms (and not polynomial features
- By default, `PolynomialFeatures()` will add a feature containing ones (a vector of `1`s) called a `bias` -- we can prevent that through the parameter `include_bias=False`
- The `degree` parameter determines the maximum number of features to create interaction terms from (in case we wanted to create an interaction term of `n_features`)
- open question whether we should apply `PolynomialFeatures` to the entire feature matrix or only to the numerical and/or categorical features and how we should do it (with everything higher degree + interaction terms or just interaction terms).
- Given the number of featues we have, we will have many more features after transforming with `PolynomialFeatures`
- The formula for calculating the number of the polynomial features is `N(n,d)=C(n+d,d)` where `n` is the number of the features, `d` is the degree of the polynomial, `C` is binomial coefficient(combination). In our case the number is `C(3+2,2)=5!/(5-2)!2!=10` but when the number of features or the degree is high the polynomial features becomes too many.

#### `sklearn.preprocessing` scalers such as `StandardScaler()`

#### Custom `helper` functions, such as `convert_df_to_numpy`
- need to create a function to convert `DataFrame` to `numpy.ndarray` then can use `FeatureUnion` to combine with `numpy.ndarray` that results from `numerical_pipeline` to form feature matrix

In [20]:
def convert_df_to_numpy(df):
    
    df = df.values
    
    return df

#### `FeatureUnion`
- to reassemble feature matrix from the product of multiple `pipelines`

### `numerical_features` Pipeline

In [21]:
numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(numerical_features)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [22]:
numerical = numerical_features_pipeline.fit_transform(X_train_all, y_train)

In [23]:
numerical.shape, len(numerical_features)

((34192, 11), 11)

### `categorical_features_pipeline`

In [24]:
categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(categorical_features)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [25]:
categorical = categorical_features_pipeline.fit_transform(X_train_all, y_train)

In [26]:
categorical.shape, len(categorical_features)

((34192, 99), 99)

### Assembling Feature Matrix via `FeatureUnion`

In [27]:
feature_matrix = FeatureUnion([('num_features', numerical_features_pipeline),
                               ('cat_features', categorical_features_pipeline)
                              ])

In [28]:
type(feature_matrix)

sklearn.pipeline.FeatureUnion

### Polynomial Logistic Regression Modeling Pipeline

In [30]:
poly_lr_model_pipe = Pipeline(steps=[
    ('get_feature_matrix', feature_matrix),
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression(penalty='l1',
                                      C=1,
                                      class_weight='balanced',
                                      solver='liblinear',
                                      random_state=0)) 
    ])

- creating a `StratifiedKFold` cross validation

In [31]:
skf = StratifiedKFold(n_splits=3,
                      shuffle=True,
                      random_state=0)

- defining `scoring_metrics`

In [32]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- conduct `StratifiedKFold` cross-validation

In [33]:
skf_results = cross_validate(poly_lr_model_pipe, # model pipeline
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1)                          

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  5.5min finished


- model run time with 3 folds ~6 minutes
- examining results

In [34]:
results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
results_summary = pd.DataFrame(results.T)
results_summary['mean_score'] = results_summary.mean(axis=1)
results_summary

,0,1,2,mean_score
test_accuracy,0.93271,0.93858,0.93525,0.93551
train_accuracy,0.98583,0.98987,0.98539,0.98703
test_f1,0.03764,0.03581,0.03403,0.03583
train_f1,0.70610,0.77061,0.69973,0.72548
test_f1_macro,0.50139,0.50205,0.50027,0.50123
train_f1_macro,0.84942,0.88271,0.84612,0.85942
test_f1_weighted,0.94935,0.95241,0.95063,0.95079
train_f1_weighted,0.98786,0.99100,0.98753,0.98880
test_precision,0.02488,0.02444,0.02281,0.02404
train_precision,0.54571,0.62682,0.53814,0.57022


#### Takeaways
- very similar results to out of the box `LogisticRegression` with no `PolynomialFeatures` transformation

### Tuning `LogisticRegression` Hyperparameters Using `GridSearchCV`
- instantiating a `Pipeline`

In [35]:
poly_lr_model_pipe = Pipeline(steps=[
    ('get_feature_matrix', feature_matrix),
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [36]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.001, 0.01, 0.10, 0.50, 1.0, 5.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- create `GridSearchCV` object

In [37]:
gs = GridSearchCV(poly_lr_model_pipe, # estimator
                  param_grid=parameter_grid, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5,
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fitting `GridSearchcV`

In [38]:
poly_log_reg_lasso = gs.fit(X_train_all,
                            y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 54.0min finished


- `GridSearchCV` takes 1 full hour to run
- examining results

In [40]:
gs_results = pd.DataFrame(poly_log_reg_lasso.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [41]:
display_cols = ['params',
                'rank_test_roc_auc',
                'mean_test_roc_auc',
                'mean_train_roc_auc']

In [42]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary

,params,rank_test_roc_auc,mean_test_roc_auc,mean_train_roc_auc
0,"{'Classifier__C': 0.01, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cl...",1,0.64919,0.81069
1,"{'Classifier__C': 0.001, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'C...",2,0.62740,0.64050
2,"{'Classifier__C': 0.1, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",3,0.56656,0.97778
3,"{'Classifier__C': 0.5, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",4,0.53539,0.99674
4,"{'Classifier__C': 1.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",5,0.52619,0.99906
5,"{'Classifier__C': 5.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",6,0.51940,1.00000


- examining features selected by `LogisticRegression` with `l1` penalty

- retrieving best performing model from `GridSearchCV`

In [43]:
poly_log_reg_lasso.best_estimator_

Pipeline(memory=None,
         steps=[('get_feature_matrix',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('num_features',
                                                 Pipeline(memory=None,
                                                          steps=[('select',
                                                                  FeatureSelector(key=['age',
                                                                                       'heightcm',
                                                                                       'weightkg',
                                                                                       'bmi',
                                                                                       'hct',
                                                                                       'creatlst',
                                                                                       'totalbumin',
    

In [44]:
poly_log_reg_lasso_best = poly_log_reg_lasso.best_estimator_.fit(X_train_all,
                                                                 y_train)

- number of features selected

In [50]:
np.count_nonzero(poly_log_reg_lasso_best['Classifier'].coef_)

290

- can retrieve the feature names from `PolynomialFeatures()` using the `get_feature_names()` method which returns a list
- `poly_features` is the named `pipeline` step

In [52]:
poly_feature_names = poly_log_reg_lasso_best['poly_features'].get_feature_names(X_train_all.columns)

- putting into a `DataFrame` for easier analysis

In [54]:
poly_log_reg_gs_features = pd.DataFrame({'poly_feature_name': poly_feature_names,
                                         'coef': poly_log_reg_lasso_best['Classifier'].coef_[0]})

- taking the absolute value so can rank

In [61]:
poly_log_reg_gs_features['abs_coef'] = abs(poly_log_reg_gs_features['coef'])

- these are the relevant features selected by the model

In [62]:
poly_lr_relevant_features = poly_log_reg_gs_features[poly_log_reg_gs_features['coef'] != 0]

In [64]:
poly_lr_relevant_features = poly_lr_relevant_features.sort_values(by=['abs_coef'],
                                                                  ascending=False)

In [65]:
poly_lr_relevant_features.shape[0]

290

In [66]:
poly_lr_relevant_features.head(25)

,poly_feature_name,coef,abs_coef
4397,cvd hdefd,0.52317,0.52317
1517,surgdt_month_Mar Tobacco_Combined,0.51059,0.51059
4474,cva numdisv_3_CORONARIES,0.40356,0.40356
310,heightcm vdinsufm_SEVERE,-0.31781,0.31781
514,bmi numdisv_3_CORONARIES,0.31084,0.31084
3170,raceblack Tobacco_Combined,-0.28885,0.28885
4960,arrhyafib numdisv_3_CORONARIES,0.28837,0.28837
2585,surgdt_DayOfWeek_Fri cvd,-0.27733,0.27733
4683,prcvint vdaort,0.27554,0.27554
133,age surgdt_DayOfWeek_Tues,0.26795,0.26795


#### Key Takeaways
- `PolynomialFeatures` transformation did not improve results of `LogisticRegression`